<a href="https://colab.research.google.com/github/sti11er/SCD/blob/main/Scene_change_detector___with_dissolve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Интеллектуальные методы обработки видео

# Ноутбук разделен на два задания:
   * ### [Эвристический SCD](#first)
   * ### [SCD с ML](#second)
   
Сроки выполнения для каждого задания — одна неделя. Оцениваются задания независимо.

## Задание 1. Scene Change Detector
<a id='first'></a>

### Обязательно к прочтению

**Внимание!**

Opencv содержит очень много высокоуровневых функций обработки изображений (например, некоторые алгоритмы компенсации движения, отслеживания объектов, распознавания образов). Использование данной библиотеки в данном задании ограничивается:
* считыванием входного видео
* преобразованием его кадров в другие цветовые пространства
* использованием свёрток Собеля

Использовать библиотеку numpy можно без ограничений.

Если вы хотите использовать функции обработки изображений и видео из другой библиотеки, то оговорите использование этой функции в чате курса.

### Описание входных данных

Выборка для тренировки лежит https://titan.gml-team.ru:5003/sharing/yX8enupJV

Данные о каждом видео лежат в файле *train_dataset\info.json*. Это список из словарей, каждый словарь содержит информацию о расположении видео, о расположении ответов на смены сцен и содержит длину видео

In [ ]:
import numpy as np
import cv2 # Для установки opencv воспользуйтесь командой в терминале conda install -c conda-forge opencv
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

In [ ]:
import json
def load_json_from_file(filename):
    with open(filename, "r") as f:
        return json.load(f, strict=False)


def dump_json_to_file(obj, filename, **kwargs):
    with open(filename, "w") as f:
        json.dump(obj, f, **kwargs)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
video_dataset = load_json_from_file('/content/drive/MyDrive/mozhet/solution_template/train_dataset/info.json')


### Загрузка видео ###

Загрузка видео осуществляется при помощи cv2.VideoCapture. Этот код изменять и дописывать не нужно.

In [ ]:
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==False:
            break
        yield frame
    cap.release()

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))

Что такое frames? Это итератор на кадры видео. Чтобы пройтись по всем кадрам последовательности, воспользуйтесь следующей конструкцией:
*Аккуратно, по одной переменной frames можно пройти только один раз!*

In [ ]:
for frame in tqdm(frames):
    pass
for frame in tqdm(frames): # Второй раз уже не будет итерации
    pass

### Пишем свой простой детектор смен сцен

На данном этапе предлагается написать простой Scene Change Detector (SCD) на основе выделения характеристик кадров, подсчёта разницы между кадрами на основе данных характеристик, а также подобрать наиболее оптимальный порог для этих признаков и совместить эти признаки.
Сменой сцен в данной задаче являются только обычные мгновенные смены сцен, без дополнительных эффектов.

В качестве примера приведён простой детектор смен, который считает межкадровую разницу между кадрами.

*Важное замечание. Здесь и далее результатом алгоритма детектора сцен являются **индексы кадров начал сцен**, при этом кадры **нумеруются с 0**. Нулевой кадр в качестве ответа указывать не нужно*

<img src="Hard_cut.jpg">

In [ ]:
def baseline_scene_change_detector(frames, threshold=2000, with_vis=False):
    """
    Baseline SCD

    Arguments:
    frames -- iterator on video frames
    threshold -- parameter of your algorithm (optional)
    with_vis -- saving neighboring frames at a scene change (optional)

    Returns:
    scene_changes -- list of scene changes (idx of frames)
    vis -- list of neighboring frames at a scene change (for visualization)
    metric_values -- list of metric values (for visualization)
    """

    def pixel_metric(frame, prev_frame):
        # Базовое расстояние между кадрами - среднеквадратическая ошибка между ними
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)

    scene_changes = []
    vis = []
    metric_values = []
    prev_frame = None
    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        if prev_frame is not None:
            # Находим расстояние между соседними кадрами
            metric_value = pixel_metric(frame, prev_frame)
            if metric_value > threshold:
                scene_changes.append(idx)
                if with_vis:
                    # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
                    if len(vis) < 100:
                        vis.append([prev_frame, frame])
            metric_values.append(metric_value)
        else:
            metric_values.append(0)
        prev_frame = frame
    return scene_changes, vis, metric_values

In [ ]:
def global_histogram_cosine_distance(cur, prev):
    gray1 = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    hist1, _ = np.histogram(gray1, bins=256, range=(0, 256))
    gray2 = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
    hist2, _ = np.histogram(gray2, bins=256, range=(0, 256))

    metric = (hist1 @ hist2) / np.linalg.norm(hist1) / np.linalg.norm(hist2)

    return metric

In [ ]:
def color_histogram(cur, prev):
    # Разделение изображений на каналы BGR
    b1, g1, r1 = cv2.split(prev)
    b2, g2, r2 = cv2.split(cur)

    # Вычисление гистограмм для каждого канала
    hist_b1, _ = np.histogram(b1, bins=256, range=(0, 256))
    hist_g1, _ = np.histogram(g1, bins=256, range=(0, 256))
    hist_r1, _ = np.histogram(r1, bins=256, range=(0, 256))
    hist_b2, _ = np.histogram(b2, bins=256, range=(0, 256))
    hist_g2, _ = np.histogram(g2, bins=256, range=(0, 256))
    hist_r2, _ = np.histogram(r2, bins=256, range=(0, 256))

    # Нормализация гистограмм
    hist_b1 = hist_b1 / np.sum(hist_b1)
    hist_g1 = hist_g1 / np.sum(hist_g1)
    hist_r1 = hist_r1 / np.sum(hist_r1)
    hist_b2 = hist_b2 / np.sum(hist_b2)
    hist_g2 = hist_g2 / np.sum(hist_g2)
    hist_r2 = hist_r2 / np.sum(hist_r2)

    # Вычисление корреляции для каждого канала
    b = np.corrcoef(hist_b1, hist_b2)[0, 1]
    g = np.corrcoef(hist_g1, hist_g2)[0, 1]
    r = np.corrcoef(hist_r1, hist_r2)[0, 1]

    metrics = [b, g, r]
    # Средняя корреляция по всем каналам
    metric = np.min(metrics)

    return metric % 250

In [ ]:
import multiprocessing as mp

def block_histogram(frame):
    # Определяем размер блока и количество блоков
    block_size = 16
    h_blocks = frame.shape[0] // block_size
    w_blocks = frame.shape[1] // block_size

    histograms = []
    similarities = []

    # Перебираем все блоки в кадре
    for i in range(h_blocks):
        for j in range(w_blocks):
            # Вырезаем блок из кадра
            block = frame[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            block = cv2.cvtColor(block, cv2.COLOR_BGR2GRAY)
            # Вычисляем гистограмму распределения яркости

            hist = cv2.calcHist([block], [0], None, [10], [0, 256])
            histograms.append(hist)
            # Если это не первый блок, сравниваем гистограмму с предыдущей
            if i > 0 or j > 0:
                #  корреляция
                method = cv2.HISTCMP_CORREL
                similarity = cv2.compareHist(histograms[-2], histograms[-1], method)
                similarities.append(similarity)

    # Вычисляем среднее значение мер подобия для всех блоков
    mean_similarity = np.mean(similarities)
    return mean_similarity

In [ ]:
from skimage.feature import hog

def hog_histogram(cur, prev, threshold):
    # Вычисление HOG для каждого изображения
    cur = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    hog1, _ = hog(prev, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
    hog2, _ = hog(cur, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)

    # Нормализация HOG
    hog1 = hog1 / np.sum(hog1)
    hog2 = hog2 / np.sum(hog2)

    # Вычисление расстояния Хеллингера
    hellinger = np.sqrt(1 - np.sqrt(hog1 * hog2).sum())

    # Сравнение расстояния с порогом
    if hellinger > threshold:
        return [1, hellinger]
    return [0, hellinger]

In [ ]:
def sifT(cur, prev, threshold):
    sift = cv2.SIFT_create()

    # Преобразуем кадр в оттенки серого
    cur = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    # Находим ключевые точки и дескрипторы в следующем кадре
    kp1, des1 = sift.detectAndCompute(cur, None)
    kp2, des2 = sift.detectAndCompute(prev, None)

    # Создаем объект BFMatcher для сопоставления дескрипторов
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Находим совпадающие пары дескрипторов
    matches = bf.match(des1, des2)

    # Сортируем пары по расстоянию
    matches = sorted(matches, key=lambda x: x.distance)

    # Вычисляем долю совпадающих пар от общего числа ключевых точек
    k = min(len(kp1), len(kp2))
    if k == 0: k = 0.01;
    ratio = len(matches) / k

    # Сравнение расстояния с порогом
    if ratio > threshold:
        return [1, ratio]
    return [0, ratio]

In [ ]:
def sobel_gradient(frame):
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
  sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
  abs_sobel_x = np.absolute(sobel_x)
  abs_sobel_y = np.absolute(sobel_y)

  sobel = np.add(abs_sobel_x, abs_sobel_y)
  return sobel

def compare_images(cur, prev, threshold):
    gradient1 = sobel_gradient(cur)
    gradient2 = sobel_gradient(image2)
    metric = np.mean((gradient1 - gradient2) ** 2)
    return metric % 250

In [ ]:
def frame_analysis(feature, cur, prev, threshold):
    metric = feature(cur, prev)
    if metric < threshold:
        return [1, metric]
    return [0, metric]

In [ ]:
prev = None
def new_frame_analysis(feature, cur, idx, threshold, with_vis, scene_changes, vis, metric_values):
    global prev
    if idx == 0:
        prev = cur

    metric = feature(cur, prev)
    if metric > threshold:
        scene_changes.append(idx)
        if with_vis:
            # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
            if len(vis) < 1000:
                vis.append([prev, cur])

    metric_values.append(metric)
    prev = cur

In [ ]:
def scene_change_detector(frames, threshold=0.8, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []
    prev_frahistme = None

    for idx, frame in tqdm(enumerate(frames), leave=False):
        if idx == 0:
            prev_frame = frame
            continue

        res = frame_analysis(global_histogram_cosine_distance, frame, prev_frame, threshold)
        if res[0]:
            scene_changes.append(idx)
            if with_vis:
                # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
                if len(vis) < 1000:
                    vis.append([prev_frame, frame])

        metric_values.append(res[1])
        prev_frame = frame

    return scene_changes, vis, metric_values

In [ ]:
from joblib import Parallel, delayed
def scene_change_detector1(frames, threshold=200, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []

    Parallel(n_jobs=8, prefer="threads")(delayed(new_frame_analysis)(global_histogram_cosine_distance, frame, idx, threshold, with_vis, scene_changes, vis, metric_values) for idx, frame in tqdm(enumerate(frames), leave=False))

    return scene_changes, vis, metric_values

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '04.mp4'))
cuts_base = load_json_from_file(os.path.join('train_dataset', 'gt', '04.json'))['cut']
scene_changes_base, vis_base, metric_values_base = scene_change_detector(frames,  with_vis=True)

Посмотрим визуально, насколько сильно алгоритм ошибается, а также на значения метрики

In [ ]:
def visualize_metric_error(frame, prev_frame, value):
    fig = plt.figure(figsize=(16,4))
    plt.suptitle('Значение метрики на текущем кадре: {:.4f}'.format(value), fontsize=24)
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(prev_frame[:,:,::-1])
    ax.set_title("Предыдущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(frame[:,:,::-1])
    ax.set_title("Текущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.subplots_adjust(top=0.80)

In [ ]:
idx = 100
visualize_metric_error(vis_base[idx][0], vis_base[idx][1], metric_values_base[scene_changes_base[idx]])
sobel_gradient(vis_base[idx][0])
sobel_gradient(vis_base[idx][1])
# смена сцен

In [ ]:
idx = 10
visualize_metric_error(vis_base[idx][0], vis_base[idx][1], metric_values_base[scene_changes_base[idx]])
# ошибается, это не смена сцен

In [ ]:
def visualize_metric_values(metric_values, threshold, cuts = None):
    sns.set()
    plt.figure(figsize=(16, 8))
    plt.plot(metric_values, label='Значение метрики на кадрах')
    plt.xlabel('Номер кадра')
    plt.ylabel('Значение метрики')
    plt.hlines(y=threshold, xmin=0, xmax=len(metric_values), linewidth=2, color='r', label='Пороговое значение')

    if cuts is not None:
        for cut in cuts:
            plt.axvline(x=cut, color='k', linestyle=':', linewidth=0.5, label='Смена сцены')

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()

In [ ]:
visualize_metric_values(metric_values_base, 0.8, cuts_base)

**Как видим, очень плохо подобран порог, да и сам признак, похоже, сильно зашумлён. Попробуйте что-то своё!**

### Ваше решение

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [ ]:
# GRADED CELL: scene_change_detector

def scene_change_detector(frames, threshold=None, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []

    ### START CODE HERE ###
    # Ваши внешние переменные
    ###  END CODE HERE  ###

    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра

        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        ###  END CODE HERE  ###
        pass

    return scene_changes, vis, metric_values

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))
cuts = load_json_from_file(os.path.join('train_dataset', 'gt', '03.json'))['cut']
scene_changes, vis, metric_values = scene_change_detector(frames, with_vis=True)

#### Обратите внимание на скорость работы алгоритма! ####
Если вычислять признаки без циклов по пикселям, а пользоваться методами из numpy, то скорость будет не медленнее 7-8 кадров в секунду.
Например, вы можете использовать функцию `np.histogram` или `cv2.calcHist` для подсчёта гистограмм, а `cv2.Sobel` для применения оператора Собеля к кадру.

In [ ]:
#Посмотрим на найденные смены сцен
idx = 1
visualize_metric_error(vis[idx][0], vis[idx][1], metric_values[scene_changes[idx]])

In [ ]:
#Посмотрим на значения метрики
visualize_metric_values(metric_values, 2000, cuts)

### Подсчёт метрики F1-Score

Чтобы оценивать алгоритм и научиться сравнивать несколько алгоритмов, нужна метрика качества. В данной задаче для оценки качества алгоритма используется F1-Score. Преимущества использования этой метрики к текущей постановке задачи смены сцен были рассказаны на лекции, напишем только формулы:
$$precision = \frac{tp}{tp+fp}$$
$$recall = \frac{tp}{tp+fn}$$
$$F = 2 * \frac{precision * recall}{precision+recall}$$

На всякий случай опишем как именно происходит подсчёт метрики для видео

1) Сначала из выборки удаляются все кадры, которые по разметке либо являются сложными переходами между сценами, либо помечены как сложные для анализа и разметки (например, титры/обилие компьютерной графики и т.п)


2) Затем для оставшихся кадров уже подсчитывается F1_Score

In [ ]:
#Эти пять клеток кода править не нужно
def calculate_matrix(true_scd, predicted_scd, scene_len, not_to_use_frames=set()):
    predicted_scd = set(predicted_scd)
    tp, fp, tn, fn = 0, 0, 0, 0
    scene_len = scene_len
    for scd in predicted_scd:
        if scd in true_scd:
            tp += 1
        elif scd not in not_to_use_frames:
            fp += 1
    for scd in true_scd:
        if scd not in predicted_scd:
            fn += 1
    tn = scene_len - len(not_to_use_frames) - tp - fp - fn
    return tp, fp, tn, fn

In [ ]:
def calculate_precision(tp, fp, tn, fn):
    return tp / max(1, (tp + fp))

In [ ]:
def calculate_recall(tp, fp, tn, fn):
    return tp / max(1, (tp + fn))

In [ ]:
def f1_score(true_scd, predicted_scd, scene_len, not_to_use_frames=set()):
    tp, fp, tn, fn = calculate_matrix(true_scd, predicted_scd, scene_len, not_to_use_frames)
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

In [ ]:
def f1_score_matrix(tp, fp, tn, fn):
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

## Придумываем признаки ##

In [ ]:
from joblib import Parallel, delayed
import pandas as pd

In [ ]:
dataset_path = '/content/drive/MyDrive/mozhet/solution_template/train_dataset'

train_data = load_json_from_file(os.path.join(dataset_path, 'info.json'))

In [ ]:
from sklearn.model_selection import train_test_split

# Разделить выборку на тренировочную и валидационную. В валидационной выборке окажется 20% всех примеров
train, test = train_test_split(train_data, test_size=0.2)

In [ ]:
train = [{'source': 'video/14.mp4', 'scene_change': 'gt/14.json', 'len': 2326},
 {'source': 'video/05.mp4', 'scene_change': 'gt/05.json', 'len': 5662},
 {'source': 'video/21.mp4', 'scene_change': 'gt/21.json', 'len': 4898},
 {'source': 'video/04.mp4', 'scene_change': 'gt/04.json', 'len': 3392},
 {'source': 'video/10.mp4', 'scene_change': 'gt/10.json', 'len': 6096},
 {'source': 'video/03.mp4', 'scene_change': 'gt/03.json', 'len': 3250},
 {'source': 'video/22.mp4', 'scene_change': 'gt/22.json', 'len': 7749},
 {'source': 'video/07.mp4', 'scene_change': 'gt/07.json', 'len': 3321}]

In [ ]:
test = [{'source': 'video/17.mp4', 'scene_change': 'gt/17.json', 'len': 2905},
 {'source': 'video/08.mp4', 'scene_change': 'gt/08.json', 'len': 3396}]

In [ ]:
def color_histogram(cur, prev):
    # Разделение изображений на каналы BGR
    b1, g1, r1 = cv2.split(prev)
    b2, g2, r2 = cv2.split(cur)

    # Вычисление гистограмм для каждого канала
    hist_b1, _ = np.histogram(b1, bins=256, range=(0, 256))
    hist_g1, _ = np.histogram(g1, bins=256, range=(0, 256))
    hist_r1, _ = np.histogram(r1, bins=256, range=(0, 256))
    hist_b2, _ = np.histogram(b2, bins=256, range=(0, 256))
    hist_g2, _ = np.histogram(g2, bins=256, range=(0, 256))
    hist_r2, _ = np.histogram(r2, bins=256, range=(0, 256))

    # Нормализация гистограмм
    hist_b1 = hist_b1 / np.sum(hist_b1)
    hist_g1 = hist_g1 / np.sum(hist_g1)
    hist_r1 = hist_r1 / np.sum(hist_r1)
    hist_b2 = hist_b2 / np.sum(hist_b2)
    hist_g2 = hist_g2 / np.sum(hist_g2)
    hist_r2 = hist_r2 / np.sum(hist_r2)

    # Вычисление корреляции для каждого канала
    b = np.corrcoef(hist_b1, hist_b2)[0, 1]
    g = np.corrcoef(hist_g1, hist_g2)[0, 1]
    r = np.corrcoef(hist_r1, hist_r2)[0, 1]

    metrics = [b, g, r]
    # Средняя корреляция по всем каналам
    metric = np.min(metrics)

    return metric

In [ ]:
def global_histogram_cosine_distance(cur, prev):
    gray1 = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    hist1, _ = np.histogram(gray1, bins=256, range=(0, 256))
    gray2 = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
    hist2, _ = np.histogram(gray2, bins=256, range=(0, 256))
[{'source': 'video/14.mp4', 'scene_change': 'gt/14.json', 'len': 2326},
 {'source': 'video/05.mp4', 'scene_change': 'gt/05.json', 'len': 5662},
 {'source': 'video/21.mp4', 'scene_change': 'gt/21.json', 'len': 4898},
 {'source': 'video/04.mp4', 'scene_change': 'gt/04.json', 'len': 3392},
 {'source': 'video/10.mp4', 'scene_change': 'gt/10.json', 'len': 6096},
 {'source': 'video/03.mp4', 'scene_change': 'gt/03.json', 'len': 3250},
 {'source': 'video/22.mp4', 'scene_change': 'gt/22.json', 'len': 7749},
 {'source': 'video/07.mp4', 'scene_change': 'gt/07.json', 'len': 3321}]
    metric = (hist1 @ hist2) / np.linalg.norm(hist1) / np.linalg.norm(hist2)

    return metric

In [ ]:
def global_histogram_euclidian_distance(cur, prev):
    gray1 = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    hist1, _ = np.histogram(gray1, bins=256, range=(0, 256))
    gray2 = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
    hist2, _ = np.histogram(gray2, bins=256, range=(0, 256))

    metric = np.linalg.norm(hist1-hist2)

    return metric

In [ ]:
def L2(cur, prev):
    metric = np.linalg.norm(cur-prev)
    return metric

In [ ]:
def block_histogram(cur, prev, size, feature, m):
    block_size = size
    h_blocks = cur.shape[0] // block_size
    w_blocks = cur.shape[1] // block_size

    metrics = []

    # Перебираем все блоки в кадре
    for i in range(h_blocks):
        for j in range(w_blocks):
            # Вырезаем блок из кадра
            block_cur = cur[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            block_prev = prev[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]

            metrics.append(feature(block_cur, block_prev))

    # Вычисляем среднее значение мер подобия для всех блоков
    if m == "mean":
        metric = np.mean(metrics)
    elif m == "median":
        metric = np.median(metrics)
    elif m == "min":
        metric = np.min(metrics)
    elif m == "max":
        metric = np.max(metrics)

    return metric

In [ ]:
def sobel_gradient(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    abs_sobel_x = np.absolute(sobel_x)
    abs_sobel_y = np.absolute(sobel_y)

    sobel = np.add(abs_sobel_x, abs_sobel_y)
    return sobel

def sobel_compare_images(cur, prev):
    gradient1 = sobel_gradient(cur)
    gradient2 = sobel_gradient(prev)
    metric = np.mean((gradient1 - gradient2) ** 2)
    return metric

In [ ]:
from types import NoneType
def ORB(cur, prev):
    # Initiate ORB detector
    orb = cv2.ORB_create()

    cur = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    # find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(cur,None)
    kp2, des2 = orb.detectAndCompute(prev,None)

    if type(des1)==NoneType or type(des2)==NoneType:
        return 0

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Match descriptors.
    matches = bf.match(des1,des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)

    # Вычисляем долю совпадающих пар от общего числа ключевых точек
    k = min(len(kp1), len(kp2))
    if k == 0: k = 1;
    ratio = len(matches) / k
    return ratio

In [ ]:
def OpticalFlow(cur, prev):
    curr_gray = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

    if type(prev_pts)==NoneType:
        return 0

    prev_pts = prev_pts.astype(np.float32)
    curr_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)

    if type(curr_pts)==NoneType:
        return 0

    curr_pts = curr_pts.astype(np.float32)

    flow = cv2.norm(curr_pts - prev_pts, cv2.NORM_L2)

    return flow

In [ ]:
def features_function(cur, prev):
    return {
        # 'global_histogram_cosine_distance': global_histogram_cosine_distance(cur, prev),
        # 'color_histogram_min': color_histogram(cur, prev),
        # 'OpticalFlow': OpticalFlow(cur, prev),
        # 'sobel_compare_images': sobel_compare_images(cur, prev),
        # 'ORB': ORB(cur, prev),
        # 'global_histogram_euclidian_distance': global_histogram_euclidian_distance(cur, prev),
        # 'L2': L2(cur, prev),
        'block_histogram': block_histogram(cur, prev, 150, global_histogram_cosine_distance, "mean")
    }

In [ ]:
def generate_table(prev, cur, cuts, idx, threshold=0.8):
    cuts = set(cuts)
    current_features = {}
    metric = features_function(cur, prev)
    current_features.update(metric)
    current_features['is_frame_change'] = 1 if idx in cuts else 0
    return current_features

In [ ]:
train_features = pd.DataFrame()

In [ ]:
test_features = pd.DataFrame()

In [ ]:
def f(n):
    yield from range(3)
    return

gen = f(3)
gen_shifted = f(3)
next(gen_shifted)
print(list(zip(gen, gen_shifted)))

In [ ]:
for video_train in tqdm(train):
    frames_train = read_video(os.path.join(dataset_path, video_train['source']))
    frames_train_shifted = read_video(os.path.join(dataset_path, video_train['source']))
    next(frames_train_shifted)
    frames_train = zip(frames_train, frames_train_shifted)
    cuts_train = load_json_from_file(os.path.join(dataset_path, video_train['scene_change']))['cut']

    train_features_cur = []

    # for idx, prev_cur in tqdm(enumerate(frames_train, 1)):
    #     prev, cur = prev_cur
    #     train_features_cur.append(generate_table(cur, prev, cuts_train, idx))

    train_features_cur = Parallel(n_jobs=2, prefer="threads")(
        delayed(generate_table)(
            prev_cur[0], prev_cur[1], cuts_train, idx) for idx, prev_cur in tqdm(enumerate(frames_train, 1)))

    train_features_cur = pd.DataFrame(train_features_cur)
    train_features = pd.concat([train_features_cur, train_features], ignore_index=True)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
for video_test in tqdm(test):
    frames_test = read_video(os.path.join(dataset_path, video_test['source']))
    frames_test_shifted = read_video(os.path.join(dataset_path, video_test['source']))
    next(frames_test_shifted)
    frames_test = zip(frames_test, frames_test_shifted)
    cuts_test = load_json_from_file(os.path.join(dataset_path, video_test['scene_change']))['cut']

    test_features_cur = []

    # for idx, prev_cur in tqdm(enumerate(frames_train, 1)):
    #     prev, cur = prev_cur
    #     train_features_cur.append(generate_table(cur, prev, cuts_train, idx))

    test_features_cur = Parallel(n_jobs=2, prefer="threads")(
        delayed(generate_table)(
            prev_cur[0], prev_cur[1], cuts_test, idx) for idx, prev_cur in tqdm(enumerate(frames_test, 1)))

    test_features_cur = pd.DataFrame(test_features_cur)
    test_features = pd.concat([test_features_cur,test_features], ignore_index=True)

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
test_features.to_csv("/content/drive/MyDrive/mozhet/test_features_block_hist.csv")

In [ ]:
train_features.to_csv("/content/drive/MyDrive/mozhet/train_features_block_hist.csv")

In [ ]:
train_features.corr()

In [ ]:
sns.displot(
    train_features,
    x="global_histogram_cosine_distance",
    hue="is_frame_change",
    stat="probability",
    common_norm=False,
    log_scale=True,
)

In [ ]:
import seaborn as sns

sns.displot(
    test_features,
    x="global_histogram_cosine_distance",
    hue="is_frame_change",
    stat="probability",
    common_norm=False,
    # log_scale=True,
)

In [ ]:
sns.boxplot(
    train_features,
    x="is_frame_change",
    y="sobel_compare_images",
    hue="is_frame_change"
)

### Тестируем разработанный метод сразу на нескольких видео

Проверим, насколько хорошо работает разработанный метод. *Учтите, что итоговое тестирование будет производиться на аналогичном, но недоступном вам наборе видео, но все параметры алгоритмов должны быть указаны вами (иными словами - подобраны на тренировочном наборе).*

In [ ]:
def run_scene_change_detector_all_video(scene_change_detector, dataset_path):
    video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
    param_log = {
        '_mean_f1_score': []
    }
    for video_info in tqdm(video_dataset, leave=False):
        # Загружаем видео, его длину и смены сцен
        frames = read_video(os.path.join(dataset_path, video_info['source']))
        video_len = video_info['len']
        true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))

        # Составляем список сцен, которые не будут тестироваться
        not_use_frames = set()
        for type_scene_change in ['trash', 'fade', 'dissolve']:
            for bad_scene_range in true_scene_changes.get(type_scene_change, []):
                not_use_frames.update(list(range(bad_scene_range[0], bad_scene_range[1] + 1)))

        predicted_scene_changes, _, _ = scene_change_detector(frames)

        param_log['f1_score_{}'.format(video_info['source'])] = f1_score(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )
        video_tp, video_fp, video_tn, video_fn = calculate_matrix(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )

        param_log['tp_{}'.format(video_info['source'])] = video_tp
        param_log['fp_{}'.format(video_info['source'])] = video_fp
        param_log['tn_{}'.format(video_info['source'])] = video_tn
        param_log['fn_{}'.format(video_info['source'])] = video_fn
        param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])
        print(param_log)

    param_log['_mean_f1_score'] = np.mean(param_log['_mean_f1_score'])
    return param_log

In [ ]:
dataset_path = 'train_dataset'

video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
param_log = {
    '_mean_f1_score': []
}
video_info = video_dataset[0]
print(video_info)

frames = read_video(os.path.join(dataset_path, video_info['source']))
video_len = video_info['len']
true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))

# Составляем список сцен, которые не будут тестироваться
not_use_frames = set()
for type_scene_change in ['trash', 'fade', 'dissolve']:
    for bad_scene_range in true_scene_changes.get(type_scene_change, []):
        not_use_frames.update(list(range(bad_scene_range[0], bad_scene_range[1] + 1)))

predicted_scene_changes, _, _ = scene_change_detector(frames)

param_log['f1_score_{}'.format(video_info['source'])] = f1_score(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )
video_tp, video_fp, video_tn, video_fn = calculate_matrix(
    true_scene_changes['cut'],
    predicted_scene_changes,
    video_len,
    not_use_frames
)


param_log['tp_{}'.format(video_info['source'])] = video_tp
param_log['fp_{}'.format(video_info['source'])] = video_fp
param_log['tn_{}'.format(video_info['source'])] = video_tn
param_log['fn_{}'.format(video_info['source'])] = video_fn
param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])

print(param_log)

In [ ]:
video_dataset = 'train_dataset'

Данная функция поможет вам посмотреть, на каких видео и на сколько ошибается ваш метод. Прогнать метод на отдельном видео и детально посмотреть кадры вы могли выше.

Кроме того, с помощью этой функции вы можете подобрать оптимальные параметры для метода.

In [ ]:
#Протестируем базовый метод
run_scene_change_detector_all_video(scene_change_detector, video_dataset)

In [ ]:
#Протестируем разработанный вами метод
run_scene_change_detector_all_video(scene_change_detector, video_dataset)

Когда вы смотрите на результат, обращайте внимание на **_mean_f1_score**  
Именно по этой метрике будет производится финальное оценивание.

## Бонусное задание: распознавание смен сцен типа "наложения"

На практике кроме катов часто встречаются смены сцен, где происходит "наложение" одной сцены на другую:

<img src="Dissolve.jpg">

### Ваше решение

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector_dissolve(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [ ]:
# GRADED CELL: scene_change_detector_dissolve

def scene_change_detector_dissolve(frames, threshold=None, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []

    ### START CODE HERE ###
    # Ваши внешние переменные
    ###  END CODE HERE  ###

    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра

        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        ###  END CODE HERE  ###
        pass

    return scene_changes, vis, metric_values

В качестве метрики качества используется видоизменённый f1-score:

Так как смена сцен не происходит за один кадр, попаданием считается попадание ответа смены сцен в отрезок, где происходит наложение.  
**Обратите внимание**, что несколько раз указывать одну смену сцен не нужно.

Попадание вне отрезков смен сцен путём наложения считается как false positive, не попадание в указанный отрезок - как false negative

In [ ]:
#Эти три клетки кода править не нужно
def calculate_matrix_dissolve(true_scd, predicted_scd, scene_len):
    predicted_scd = set(predicted_scd)
    tp, fp, tn, fn = 0, 0, 0, 0
    scene_len = scene_len
    checked_dissolve_segments = set()
    total_scene_dissolve_len = np.sum([dissolve_segment[1] - dissolve_segment[0] + 1 for dissolve_segment in true_scd])
    for scd in predicted_scd:
        for dissolve_segment in true_scd:
            if scd in range(dissolve_segment[0], dissolve_segment[1] + 1):
                if tuple(dissolve_segment) not in checked_dissolve_segments:
                    tp += 1
                    checked_dissolve_segments.add(tuple(dissolve_segment))
                break
        else:
            fp += 1
    fn = len(true_scd) - len(checked_dissolve_segments)
    tn = scene_len - total_scene_dissolve_len + len(true_scd) - tp - fp - fn
    return tp, fp, tn, fn

In [ ]:
def f1_score_dissolve(true_scd, predicted_scd, scene_len):
    tp, fp, tn, fn = calculate_matrix_dissolve(true_scd, predicted_scd, scene_len)
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

In [ ]:
def run_scene_change_detector_all_video_dissolve(scene_change_detector, dataset_path):
    video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
    param_log = {
        '_mean_f1_score': []
    }
    for video_info in tqdm(video_dataset, leave=False):
        frames = read_video(os.path.join(dataset_path, video_info['source']))
        video_len = video_info['len']
        true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))

        predicted_scene_changes, _, _ = scene_change_detector(frames)
        param_log['f1_score_{}'.format(video_info['source'])] = f1_score_dissolve(
            true_scene_changes.get('dissolve', []),
            predicted_scene_changes,
            video_len
        )
        video_tp, video_fp, video_tn, video_fn = calculate_matrix_dissolve(
            true_scene_changes.get('dissolve', []),
            predicted_scene_changes,
            video_len
        )
        param_log['tp_{}'.format(video_info['source'])] = video_tp
        param_log['fp_{}'.format(video_info['source'])] = video_fp
        param_log['tn_{}'.format(video_info['source'])] = video_tn
        param_log['fn_{}'.format(video_info['source'])] = video_fn
        param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])
    param_log['_mean_f1_score'] = np.mean(param_log['_mean_f1_score'])
    return param_log

In [ ]:
video_dataset_path = 'train_dataset'

In [ ]:
#Протестируем разработанный вами метод
run_scene_change_detector_all_video_dissolve(scene_change_detector_dissolve, video_dataset_path)

### Немного об оценивании задания

Оценивание задания будет производиться по следующей схеме:  

Пусть на скрытой выборке по F-метрике вы получили X, лучшее решение получило Y.

1. Базовая часть оценивется как $$20 * \left(\frac{\max(0, X_{base} - 0.5)}{Y_{base} - 0.5}\right)^2 + Bonus_{base}$$ Бонусные баллы $Bonus$ можно получить за оригинальные идеи в задаче или в её реализации
2. Дополнительное задание оценивается как $$5 * \frac{\max(0, X_{add} - 0.1)}{Y_{add} - 0.1} + Bonus_{add}$$Процесс получения бонусных баллов аналогичен получению бонусных баллов в базовой части

### Ваши ощущения ##

*До дедлайна пару часов и вы никак не можете улучшить текущее решение? Или наоборот, вы всё сделали очень быстро? Опишите кратко ваши ощущения от задания - сколько времени вы потратили на задание, сколько вы потратили на изучение питона и установку необходимых библиотек, как быстро вы придумывали новые идеи и как они давали прирост по метрике и в целом насколько это задание вам понравилось и что хотели бы изменить/добавить.*

<a id='second'></a>

## Задание 2. Scene change detector. Машинное обучение

**Внимание!**

В этом задании можно использовать все, что разрешалось в Задании №1, а также библиотеки:
* pandas
* sklearn

Большинство функций, использующихся в этом задании, реализованы выше.

### Бейзлайн

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import pickle

Обучим простой SVM классификатор над метрикой попиксельной разницы кадров на нескольких видео. Воспользуемся функцией из первого задания

In [ ]:
def get_train_data(train_videos):
    X_train, y_train = np.array([]), np.array([])
    for video in train_videos:
        frames = read_video(os.path.join('train_dataset', 'video', f'{video}.mp4'))
        # baseline функция попиксельного сравнения кадров из прошлого задания
        # нам нужны не сами смены сцен, а только значения метрик
        _, _, metric_values = baseline_scene_change_detector(frames)

        cuts = load_json_from_file(os.path.join('train_dataset', 'gt', f'{video}.json'))['cut']
        video_scenes = np.array([0 for i in range(len(metric_values))])
        video_scenes[cuts] += 1

        # добавляем в разметку текущее видео
        X_train = np.hstack((X_train, metric_values))
        y_train = np.hstack((y_train, video_scenes))

    return X_train, y_train

In [ ]:
train_videos = ['04', '05']
train_X, train_y = get_train_data(train_videos)

In [ ]:

train_videos = ['08']
test_X, test_y = get_train_data(train_videos)

In [ ]:
train_X = train_features.drop('is_frame_change', axis=1)
train_y = train_features['is_frame_change']

In [ ]:
train_features[train_features.is_frame_change == 1]

In [ ]:
train_X

In [ ]:
test_X

In [ ]:
test_X = test_features.drop('is_frame_change', axis=1)
test_y = test_features['is_frame_change']

In [ ]:
clf = RandomForestClassifier()
# params = {}
# clf = GridSearchCV(rfc, params)

In [ ]:
a.shape, b.shape, a_ans.shape, b_ans.shape

In [ ]:
from sklearn.metrics import f1_score, precision_score, accuracy_score

In [ ]:
a, b, a_ans, b_ans = train_test_split(train_X, train_y, test_size=0.7, stratify=train_y)
rf = RandomForestClassifier(max_depth=None, class_weight='balanced', n_estimators=100, bootstrap=True, min_samples_split=5, criterion='log_loss')
# rf = DecisionTreeClassifier(max_depth=None)
# rf = SVC(class_weight='balanced')
rf.fit(a, a_ans)
preds = rf.predict(b)
f1_score(b_ans, preds)

In [ ]:
accuracy_score(b_ans, preds)

In [ ]:
b_ans.sum()

In [ ]:
np.sum(preds)

In [ ]:
np.sum((b_ans == preds) & (b_ans == 1))

In [ ]:
f1_score(a_ans, rf.predict(a))

In [ ]:
accuracy_score(a_ans, rf.predict(a))

In [ ]:
a_ans.sum()

In [ ]:
rf.predict(a).sum()

In [ ]:
from sklearn.tree import plot_tree
plot_tree(rf)
plt.show()

In [ ]:
clf.fit(train_X, train_y)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# Определим метод кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Выполним кросс-валидацию
scores = cross_val_score(clf, train_X, train_y, cv=kf)

# Выведем результаты
print(f"Точность на каждом фолде: {scores}")
print(f"Средняя точность: {scores.mean():.2f}")

In [ ]:
params = {}
clf = GridSearchCV(rfc, params)
clf.fit(train_X, train_y)

In [ ]:
params = {"kernel": "rbf", "C": 1}
params = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(svc, params)

In [ ]:
train_Y = clf.predict(train_X)
test_Y = clf.predict(test_X)

In [ ]:
test_y.shape

In [ ]:
np.sum(test_y)

In [ ]:
len(train_features[train_features.is_frame_change == True])

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '17.mp4'))
frames = list(frames)
cuts_base = load_json_from_file(os.path.join('train_dataset', 'gt', '17.json'))['cut']

In [ ]:
def visualize_metric_error(frame, prev_frame, value):
    fig = plt.figure(figsize=(16,4))
    plt.suptitle('Значение метрики на текущем кадре: {:.4f}'.format(value), fontsize=24)
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(prev_frame[:,:,::-1])
    ax.set_title("Предыдущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(frame[:,:,::-1])
    ax.set_title("Текущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.subplots_adjust(top=0.80)

In [ ]:
id = 0

In [ ]:
visualize_metric_error(frames[id], frames[id-1], train_X.iloc[id].sobel_compare_images)
print(train_y[id])

In [ ]:
from sklearn.metrics import f1_score, precision_score

In [ ]:
print('F1-Score на тренировочной выборке', f1_score(train_y, train_Y))
print('F1-Score на контрольной выборке', f1_score(test_y, test_Y))

In [ ]:
print('precision_score на тренировочной выборке', precision_score(train_y, train_Y))
print('precision_score на контрольной выборке', precision_score(test_y, test_Y))

In [ ]:
clf.feature_names_in_, clf.feature_importances_
plt.bar(clf.feature_names_in_, clf.feature_importances_)
plt.show()

In [ ]:
# создание модели
# подберите лучшие параметры для данной задачи
params = {"kernel": "rbf", "C": 1}
model = SVC(**params)
model.fit(X_train.reshape(-1, 1), y_train)

 Сохраним модель в файле *model.pkl*

In [ ]:
pickle.dump(clf, open("model.pkl", "wb"))

Посмотрим как модель работает на тестовых видео

Обратите внимание на то, что внутри функции модель загружается из памяти из файла *model.pkl*

In [ ]:
def baseline_scene_change_detection_ml(frames):
    # # подготавливаем данные для видео
    # _, _, metric_values = scene_change_detector(frames)
    # X_test = np.array(metric_values).reshape(-1, 1)

    # загружаем модель и делаем предсказания
    model = pickle.load(open("model.pkl", 'rb'))
    predict_cuts = model.predict(test_X)

    print(test_X)

    return np.where(predict_cuts > 0)[0], [], test_X

In [ ]:
def run_scene_change_detector_ml_one_video(scene_change_detector, dataset_path, video_num):
    video_info = load_json_from_file(os.path.join(dataset_path, 'info.json'))[video_num]

    # Загружаем видео, его длину и смены сцен
    frames = read_video(os.path.join(dataset_path, video_info['source']))
    video_len = video_info['len']
    true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))

    # Составляем список сцен, которые не будут тестироваться
    not_use_frames = set()
    for type_scene_change in ['trash', 'fade', 'dissolve']:
        for bad_scene_range in true_scene_changes.get(type_scene_change, []):
            not_use_frames.update(list(range(bad_scene_range[0], bad_scene_range[1] + 1)))

    predicted_scene_changes, _, _ = scene_change_detector(frames)

    return f1_score(
        true_scene_changes['cut'],
        predicted_scene_changes,
        video_len,
        not_use_frames
    )

Посчитаем F1 score для одного видео:

In [ ]:
video_num = 9
run_scene_change_detector_ml_one_video(baseline_scene_change_detection_ml, 'train_dataset', video_num)

### Ваше решение

Чтобы использовать свою обученную модель при отправке решения, необходимо сохранить ее через пакет pickle в файл model.pkl и отправить его вместе с jupyter ноутбуком.
Этот файл вы можете открывать и использовать прямо в функции вашего решения

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector_dissolve(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [ ]:
# GRADED CELL: scene_change_detector_ml

def scene_change_detector_ml(frames, with_vis = False):
    scene_changes = []
    vis = []
    metric_values = []

    ###
    #  опишите здесь все функции, нужные для вашего решения
    ###


    ### START CODE HERE ###
    # Ваши внешние переменные
    ###  END CODE HERE  ###

    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра

        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        ###  END CODE HERE  ###
        pass

    model = pickle.load(open("model.pkl", 'rb'))
    predict_cuts = model.predict(X_test)

    return np.where(predict_cuts == 1)[0], vis, metric_values

Проверим ваше решение на всех видео.

Не забывайте о том, что при итоговой оценке решений будет использоваться другой набор видео. Не переобучите модель!

In [ ]:
video_dataset_path = 'train_dataset'

In [ ]:
def run_scene_change_detector_all_video(scene_change_detector_ml, video_dataset_path):
    for i in range(1, 10):
        t = run_scene_change_detector_ml_one_video(scene_change_detector_ml, video_dataset_path, i)
        print(i, t)

In [ ]:
run_scene_change_detector_all_video(baseline_scene_change_detection_ml, video_dataset_path)

**Советы**

* Используйте кросс-валидацию
* Подумайте как лучше разделять видео на тренировочную и тестовые выборки
* Подбирайте параметры модели (в библиотеке sklearn есть метод GridSearchCV для автоматического подбора параметров)
* Пробуйте разные методы машинного обучения (из sklearn)

## Бонусное задание: детектор смен сцен типа наложение

Аналогично детектору из задания №1 за исключением того, что можно (и нужно) использовать машинное обучение:)

In [ ]:
# GRADED CELL: scene_change_detector_dissolve_ml

def scene_change_detector_dissolve_ml(frames, threshold=None, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []

    ### START CODE HERE ###
    # Ваши внешние переменные
    ###  END CODE HERE  ###

    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра

        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        ###  END CODE HERE  ###
        pass

    return scene_changes, vis, metric_values

In [ ]:
video_dataset_path = 'train_dataset'
#Протестируем разработанный вами метод
run_scene_change_detector_all_video_dissolve(scene_change_detector_dissolve, video_dataset_path)

### Ваши ощущения ##

*Как и в первой части интересно узнать какие моменты показались простыми, а какие сложными. Много ли времени ушло на изучение sklearn и методов машинного обучения. Дало ли машинное обучение сразу прирост по сравнению с эврестической частью? Как вы контролировали переобучение?*